In [1]:
#Import the libraries

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import tensorflow
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tflearn
import json
import numpy
import random
import pickle

C:\Users\Jadon\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Instructions for updating:
non-resource variables are not supported in the long term


C:\Users\Jadon\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [9]:
#Loading Dataset

with open('./intents.json') as file:
    data=json.load(file)
#print(data)
#print(data['intents'][0])

JSONDecodeError: Expecting ',' delimiter: line 55 column 68 (char 2873)

In [3]:
#Processing Data
#rb -- read bytes
try:
    with open("data.pickle","rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words=[]
    labels=[]
    docs_x=[]
    docs_y=[]

    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds=nltk.word_tokenize(pattern) 
            words.extend(wrds)
            docs_x.append(wrds)
            #print(docs_x)
            docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])
    #print(labels)

    #Remove duplicates from the list words

    words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
    words = sorted(list(set(words)))
    #print(words)

    #Sort labels
    labels = sorted(labels)

    #Neural network understands only numbers and not strings

    training=[]
    output=[]

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag=[]

        wrds = [stemmer.stem(w.lower()) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])]=1

        training.append(bag)
        output.append(output_row)

    #Building the model

    training=numpy.array(training)
    #print(training)
    output=numpy.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)

In [4]:
#Classifying our data

#input layer
net = tflearn.input_data(shape=[None,len(training[0])])
print(len(training[0]))
print(len(output[0]))
#two hidden layers
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)

#output layer
net = tflearn.fully_connected(net,len(output[0]),activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

#Training the model
try: 
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000,batch_size=8, show_metric=True)
    model.save("model.tflearn")


57
12
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Jadon\Deep Learning Projects\Chatbot\model.tflearn


In [7]:
#Predictions 

def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]
    
    s_words=nltk.word_tokenize(s)
    s_words=[stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w==se:
                bag[i]=1
    return numpy.array(bag)

In [8]:
def chat():
    print("Start talking with the bot(enter quit to exit)")
    while True:
        inp = input("You: ")
        if inp.lower() =="quit":
            break
        
        results=model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
        
        print(random.choice(responses))

chat()

Start talking with the bot(enter quit to exit)
You: hello


NameError: name 'data' is not defined